<a href="https://colab.research.google.com/github/nureeee/DeepLearning/blob/main/residual_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1, 1), padding='same')

    def call(self, x, training=False):
        h = self.bn1(x, training=training) # 훈련에서는 true, 추론에서는false
        h = tf.nn.relu(h)
        h = self.conv1(h)

        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)

        return self.identity(x) + h


In [4]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        
        self.sequence = list()

        for f_in, f_out in zip([filter_in] + list(filters), filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
    
    def call(self, x, training=False):
        for unit in self.sequence:
            x = unit(x, training=False)
        return x

# 모델 정의

In [12]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu')

        self.res1 = ResnetLayer(8, (16, 16), (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.res2 = ResnetLayer(16, (32, 32), (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.res3 = ResnetLayer(32, (64, 64), (3, 3))

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False):
        x = self.conv1(x)

        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)

        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)

        return x

In [13]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255., X_test / 255.
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [14]:
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [15]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [16]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

@tf.function
def test_step(model, images, labels, losss_object, optimizer, test_loss, test_accuracy):
  # inference 단계
  # 추론 단계에서는 BN에 대한 학습이 있으면 안됨!
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)



In [17]:
# 모델 생성
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer   = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [18]:
for epoch in range(10):

  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_image, test_labels in test_ds:
    test_step(model, images, labels, loss_object, optimizer, test_loss, test_accuracy)

  print("Epoch : {}, Train Loss : {:.6f}, Train Acc : {:.6f}, Test Loss : {:.6f}, Test Acc : {:.6f}".format(
      epoch+1,
      train_loss.result(),
      train_accuracy.result() * 100,

      test_loss.result(),
      test_accuracy.result() * 100
  ))

  # loss, accuracy 평가를 1에폭마다 초기화 하면서 보기
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch : 1, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.010477, Test Acc : 100.000000
Epoch : 2, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.008617, Test Acc : 100.000000
Epoch : 3, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.008484, Test Acc : 100.000000
Epoch : 4, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.000797, Test Acc : 100.000000
Epoch : 5, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.000249, Test Acc : 100.000000
Epoch : 6, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.000784, Test Acc : 100.000000
Epoch : 7, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.000075, Test Acc : 100.000000
Epoch : 8, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.006090, Test Acc : 100.000000
Epoch : 9, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.003477, Test Acc : 100.000000
Epoch : 10, Train Loss : 0.000000, Train Acc : 0.000000, Test Loss : 0.001989, Test Acc : 100.000000